# Project Solution - Primary

You have to work on the [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view) dataset. Some information on the datasets are on the [Kaggle](https://www.kaggle.com/gaborfodor/additional-kiva-snapshot) web page.

## Basic tasks

**1.** Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one `lender`.

Firstly, csv file is read and data frame with unnormalized data is created:

In [1]:
import pandas as pd
import numpy as np
import time as tm
FINAL = True # This is final calculation, on complete data
start_time = tm.time()
loans_lenders_un = pd.read_csv('../../Datasets/kiva/loans_lenders.csv')
if not FINAL:
    loans_lenders_un = loans_lenders_un.head(10000) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lenders_un

--- 4.1969993114471436 seconds ---


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


Data frame with structured data will be created with help of intermediate object - list of pairs (loan_id, lender) packed in the dictionary:

In [ ]:
import time as tm
start_time = tm.time()
lis = []
for index, row in loans_lenders_un.iterrows(): 
    ls = row['lenders'].split(',')
    for l in ls:
        lis.append({ 'loan_id' : row['loan_id'], 'lender': l.strip() })
loans_lender = pd.DataFrame(lis) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lender 

**2.** For each loan, add a column `duration` corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Firstly, data frame should be loaded and structure of the data frame `loans` should be determined:

In [ ]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
loans = pd.read_csv('../../Datasets/kiva/loans.csv')
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans.columns

After thar, values from the data frame should be displayed:

In [ ]:
if not FINAL:
    loans = loans.head(20000)
loans

We are now mainly interested in columns 'planned_expiration_time', 'disburse_time':

In [ ]:
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time'] ]
loans_sel

In [ ]:
from datetime import datetime as dt
import time as tm
start_time = tm.time()
loans['duration'] = None
for index, row in loans.iterrows(): 
    s2 = row['planned_expiration_time']
    s1 = row['disburse_time']
    try:
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    try:
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    loans.loc[index,'duration'] = (d2 - d1).days 
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time', 'duration'] ]
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_sel

**3.** Find the lenders that have funded at least twice.

- Those are lenders that are duplicated in structured loan_leneders dataframe.


In [ ]:
import time as tm
start_time = tm.time()
loans_lender_2_more = loans_lender[loans_lender.duplicated(['lender'])]
lenders_funded_2_more = list(set(loans_lender_2_more['lender']))
print("--- %s seconds ---" % (tm.time()-start_time)) 
lenders_funded_2_more

**4.** For each country, compute how many loans have involved that country as borrowers.

- This solution is based on panda grouping:

In [ ]:
import time as tm
start_time = tm.time()
loans_count = loans.groupby(['country_code','country_name'], as_index=False)[['loan_id']].count()
loans_count.columns = ['country_code','country_name','loan_id_count']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_count

**5.** For each country, compute the overall amount of money borrowed.


- This solution is based on panda grouping:

In [ ]:
import time as tm
start_time = tm.time()
loans_sum = loans.groupby(['country_code','country_name'], as_index = False)[['loan_amount']].sum()
loans_sum.columns = ['country_code','country_name','loan_amount_sum']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_sum

**6.** Like the previous point, but expressed as a percentage of the overall amount lent.


- This solution is based on panda functions:

In [ ]:
import time as tm
start_time = tm.time()
loans_total = loans['loan_amount'].sum()
loans_percent = loans.groupby(['country_code','country_name'], as_index = False)[['loan_amount']].sum()
loans_percent.columns = ['country_code','country_name','loan_amount_sum']
loans_percent['percent'] = loans_percent['loan_amount_sum']/loans_total*100
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_percent

**7.** Like the three previous points, but split for each year (with respect to `disburse time`).


- This solution is based on panda functions:

In [ ]:
import time as tm
start_time = tm.time()
from datetime import datetime as dt
loans_total = loans['loan_amount'].sum()
loans['disburse_time_year'] = None
for index, row in loans.iterrows():
    try:
        y = dt.strptime(row['disburse_time'], "%Y-%m-%d %H:%M:%S.%f %z").year
    except TypeError:
        y = None
    except ValueError:
        y = None
    loans.loc[index,'disburse_time_year'] = y 
loans_agg = loans.groupby(['country_code','country_name','disburse_time_year'\
                          ], as_index = False).agg({'loan_id':['count'],\
                                                    'loan_amount':['sum', lambda x: x.sum()/loans_total*100]})
loans_agg.columns = ['country_code','country_name','disburse_time_year','loan_id_count', \
                     'loan_amount_sum','loan_amount_percent']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_agg

**8.** For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


- This solution is based on panda functions:

In [ ]:
import time as tm
start_time = tm.time()
merged = pd.merge(left=loans_lender,right=loans, left_on='loan_id', right_on='loan_id')
merged = merged[['loan_id','lender', 'loan_amount']]
number_of_lenders_by_loan_id = merged.groupby(['loan_id'], as_index = False)[['lender']].count()
number_of_lenders_by_loan_id
merged2 = pd.merge(left=merged,right=number_of_lenders_by_loan_id, left_on='loan_id', right_on='loan_id')
merged2['precise_amount'] = merged2.loan_amount / merged2.lender_y
merged2
lent_by_lender = merged2.groupby(['lender_x'], as_index = False)[['precise_amount']].sum()
print("--- %s seconds ---" % (tm.time()-start_time)) 
lent_by_lender
#result.loc[result['lender_x'] == 'muc888'] # result for one lender

**9.** For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [ ]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
lenders = pd.read_csv('../../Datasets/kiva/lenders.csv')
if not FINAL:
    lenders = lenders.head(20000) 
lender_states = lenders[['permanent_name','country_code']]
lender_states

In [ ]:
lenders_in_states = lender_states.loc[lender_states['country_code'].notnull()]
lenders_in_states

In [ ]:
lenders_without_states = lender_states.loc[lender_states['country_code'].isnull()]
lenders_without_states

In [ ]:
lent_and_state = pd.merge(left=lent_by_lender, right=lenders_in_states, left_on='lender_x', right_on='permanent_name')
lent_total = lent_and_state['precise_amount'].sum()
lent_by_state = lent_and_state.groupby(['country_code'], as_index = False\
                                      ).agg({'precise_amount':['count', 'sum', \
                                                               lambda x: x.sum()/lent_total]})
lent_by_state.columns = ['country_code', 'lent_count', 'lent_sum', 'lent_factor']
lent_by_state

In [ ]:
lent_no_state = pd.merge(left=lent_by_lender, right=lenders_without_states, \
                         left_on='lender_x', right_on='permanent_name')
lent_no_state_total = lent_no_state['precise_amount'].sum()
lent_no_state_total

In [ ]:
lent_by_state['additional_lent'] = lent_by_state.lent_factor * lent_no_state_total
lent_by_state

In [ ]:
loan_and_state = loans[['loan_id','country_code','loan_amount']]
loan_and_state

In [ ]:
loan_by_state = loan_and_state.groupby(['country_code'], as_index = False).agg({'loan_amount':['sum']})
loan_by_state.columns = ['country_code','loan_amount_sum']
loan_by_state

In [ ]:
loan_and_lent = pd.merge(loan_by_state, lent_by_state, how='left', on=['country_code'])
loan_and_lent['difference'] = loan_and_lent.loan_amount_sum - loan_and_lent.lent_sum - loan_and_lent.additional_lent 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loan_and_lent[loan_and_lent.difference == loan_and_lent.difference.max()]

**10.** Which country has the highest ratio between the difference computed at the previous point and the population?


In [ ]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
country_stat = pd.read_csv('../../Datasets/kiva/country_stats.csv')
country_stat = country_stat[['country_code', 'country_name', 'population', 'population_below_poverty_line']]
country_stat

In [ ]:
country_stat_loan_and_lent = pd.merge(left=country_stat, right=loan_and_lent, \
                                      left_on='country_code', right_on='country_code')
country_stat_loan_and_lent = country_stat_loan_and_lent[['country_code', 'country_name', 'difference', \
                                                         'population', 'population_below_poverty_line']]
country_stat_loan_and_lent['ratio_1'] = country_stat_loan_and_lent.difference/country_stat_loan_and_lent.population
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat_loan_and_lent[country_stat_loan_and_lent.ratio_1 == country_stat_loan_and_lent.ratio_1.max()]

**11.** Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [ ]:
import time as tm
start_time = tm.time()
country_stat_loan_and_lent = pd.merge(left=country_stat, right=loan_and_lent, \
                                      left_on='country_code', right_on='country_code')
country_stat_loan_and_lent = country_stat_loan_and_lent[['country_code', 'country_name', 'difference', \
                                                         'population', 'population_below_poverty_line']]
country_stat_loan_and_lent['ratio_2'] = country_stat_loan_and_lent.difference\
/(country_stat_loan_and_lent.population * ((100-country_stat_loan_and_lent.population_below_poverty_line)/100) )
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat_loan_and_lent[country_stat_loan_and_lent.ratio_2 == country_stat_loan_and_lent.ratio_2.max()]

**12.** For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [ ]:
import time as tm
start_time = tm.time()

from datetime import datetime as dt, timezone as tz

now = dt.now(tz.utc)

def days_in_year(year):
    d1 = dt.strptime( str(year)+'-01-01', "%Y-%m-%d") 
    d1 = d1.replace(tzinfo=None)
    d2 = dt.strptime( str(year)+'-12-31', "%Y-%m-%d")
    d2 = d2.replace(tzinfo=None)
    return (d2-d1).days+1

def total_amount_of_loans(disb, plan_exp, amount):
    if( disb.year == plan_exp.year ):
        return [(disb.year,amount)]
    disb = disb.replace(tzinfo=None)
    plan_exp = plan_exp.replace(tzinfo=None) 
    total_days = (plan_exp-disb).days 
    total_days += 1
    total_amount = []
    curr = disb
    curr = curr.replace(tzinfo=None)
    for y in range(disb.year, plan_exp.year):
        dy = (dt.strptime(str(y)+'-12-31',"%Y-%m-%d").replace(tzinfo=None)-curr).days + 1
        total_amount.append((y, dy * amount / total_days))
        curr = dt.strptime(str(y+1)+'-01-01',"%Y-%m-%d")
        curr = curr.replace(tzinfo=None)
    dy = (plan_exp-curr).days + 1
    total_amount.append( (plan_exp.year, dy * amount / total_days) )
    return total_amount

d_t = dt.strptime( '2016-12-01', "%Y-%m-%d")
p_e_t = dt.strptime( '2018-01-30', "%Y-%m-%d")
total_amount_of_loans(d_t, p_e_t, 5000)

In [ ]:
import pandas as pd
import numpy as np
loans
loans_base = loans[['loan_id','loan_name','loan_amount','planned_expiration_time','disburse_time']]
loans_base

In [ ]:
lis = []
for index, row in loans_base.iterrows():
    s2 = row['planned_expiration_time']
    try:
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    s1 = row['disburse_time']
    try:
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    try:
        amt = float(row['loan_amount'])
    except ValueError:
        continue
    except TypeError:
        continue
    ls = total_amount_of_loans(d1, d2, amt)
    for l in ls:
        lis.append({ 'loan_id' : row['loan_id'], 'loan_name': row['loan_name'], 'loan_amount': row['loan_amount'], \
                    'planned_expiration_time': row['planned_expiration_time'], \
                    'disburse_time': row['disburse_time'],'year': l[0], 'ammount':l[1] })
loans_final = pd.DataFrame(lis) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_final